In [96]:
import pywt
# (cA, cD) = pywt.dwt([1, 2, 3, 4, 5, 6], 'db1')
# ([2.12132034 4.94974747 7.77817459], [-0.70710678 -0.70710678 -0.70710678])
# https://pywavelets.readthedocs.io/en/latest/ref/dwt-discrete-wavelet-transform.html
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
# https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html
from matplotlib import style
# style.use('ggplot')

In [8]:
df = pd.read_csv("filtered_chocolate/0.csv", usecols=['ch0', 'ch1', 'ch2', 'ch3'])
print(df.loc[0:14].values)
dwt(df.loc[0:14])

[[4.95072450e-05 1.79903582e-04 3.89218330e-05 2.74255870e-05]
 [4.82313330e-05 1.79841183e-04 3.02605330e-05 2.27624550e-05]
 [4.26322220e-05 1.73451379e-04 3.33227220e-05 2.34134500e-05]
 [4.26322220e-05 1.73451379e-04 3.33227220e-05 2.34134500e-05]
 [4.20603900e-05 1.70729123e-04 3.85809690e-05 2.34134500e-05]
 [3.95718960e-05 1.65071338e-04 3.25217840e-05 2.44528060e-05]
 [3.95718960e-05 1.65071338e-04 3.25217840e-05 2.44528060e-05]
 [3.29511240e-05 1.60115771e-04 3.63439320e-05 2.44528060e-05]
 [3.19154930e-05 1.57691538e-04 3.45185400e-05 2.42860990e-05]
 [2.77431680e-05 1.52460299e-04 2.92453910e-05 1.32350250e-05]
 [2.40607190e-05 1.48248859e-04 2.64681880e-05 1.32350250e-05]
 [2.32327730e-05 1.45413913e-04 1.76522880e-05 3.86871400e-06]
 [2.16225160e-05 1.42143108e-04 1.85566020e-05 3.86871400e-06]
 [2.20388170e-05 1.42300501e-04 1.94925810e-05 4.70504200e-06]
 [2.26655970e-05 1.43103302e-04 1.81552020e-05 4.75347000e-06]]


array([ 6.0675650e-07,  1.9990840e-06,  0.0000000e+00,  0.0000000e+00,
       -1.5846455e-06,  3.5492705e-06,  8.3260250e-07, -1.9110740e-06,
       -5.2945700e-07, -2.8889625e-06, -1.0035000e-06, -2.7520550e-07,
       -1.2945400e-07, -4.9825500e-08,  0.0000000e+00,  0.0000000e+00])

In [25]:
df = pd.read_csv("filtered_pasta/0.csv", usecols=['ch0', 'ch1', 'ch2', 'ch3'])
arr = df.loc[0:14].div(0.00001)["ch0"].values
print(arr)
print(type(arr))

arr = np.append(arr, arr)
print(arr)
# dwt(df.loc[0:14].values)
print(raw(df.loc[0:14]))

[5.2820891 4.7696754 4.9255788 4.5990571 4.2547472 4.0634535 3.1762756
 2.2802502 2.4748035 2.3787841 2.1195039 1.7683953 1.9039027 1.9016676
 1.7171726]
<class 'numpy.ndarray'>
[5.2820891 4.7696754 4.9255788 4.5990571 4.2547472 4.0634535 3.1762756
 2.2802502 2.4748035 2.3787841 2.1195039 1.7683953 1.9039027 1.9016676
 1.7171726 5.2820891 4.7696754 4.9255788 4.5990571 4.2547472 4.0634535
 3.1762756 2.2802502 2.4748035 2.3787841 2.1195039 1.7683953 1.9039027
 1.9016676 1.7171726]
[ 5.2820891  4.7696754  4.9255788  4.5990571  4.2547472  4.0634535
  3.1762756  2.2802502  2.4748035  2.3787841  2.1195039  1.7683953
  1.9039027  1.9016676  1.7171726 18.5689889 18.1193464 18.1288458
 17.9178081 17.1777792 16.4050609 15.7933682 15.1385553 15.0495209
 14.8945488 14.3660232 14.42004   14.6069564 14.4448131 14.2258592
  4.8656017  5.0429255  4.3747015  3.8227066  4.4032931  3.9415434
  3.1800009  2.8452836  2.2577122  2.2732653  1.9093044  2.0206906
  2.1368265  1.9024126  1.9686297  5.2035786  4

In [129]:
def wt(segment):
    # featureSize = 8
    return pywt.wavedec(segment["ch0"], 'db1', level=3)[2]

def dwt(segment):
    # featureSize = 16
    coeffs = pywt.dwt2(segment.values, 'haar')
    cA, (cH, cV, cD) = coeffs
    res = cA.flatten()
    return res

def raw(segment):
    segment = segment.div(0.00001)
    # featureSize = 4 * batchSize + 16
    res = segment["ch0"].values
    res = np.append(res, segment["ch1"].values)
    res = np.append(res, segment["ch2"].values)
    res = np.append(res, segment["ch3"].values)
    res = np.append(res, dwt(segment))
    return res

featureSize = 76
def getFeature(segment):
    return raw(segment)

def featuresFromFile(fileName, fileNum):
    global featureSize
    batch = 15
    df = pd.read_csv("filtered_" + fileName + "/" + str(fileNum) + ".csv", usecols=['ch0', 'ch1', 'ch2', 'ch3'])
    pickups = np.empty(shape=[0,featureSize])
    putdowns = np.empty(shape=[0,featureSize])

    rows = df.shape[0]
    turns = rows / batch
    turns -= turns % 2
    rows = turns * batch
    r = 0
    while r < rows:
        # pick-up
        pickups = np.vstack((pickups, getFeature(df.loc[r:r+batch-1])))
        r += batch
        # put-down
        putdowns = np.vstack((putdowns, getFeature(df.loc[r:r+batch-1])))
        r += batch
    return (pickups, putdowns)

def labelFeatures(pickupFeatures, pickupLabels, putdownFeatures, putdownLabels, fileName, fileNum, label):
    (pickups, putdowns) = featuresFromFile(fileName, fileNum)
    # register features
    pickupFeatures = np.vstack((pickupFeatures, pickups))
    putdownFeatures = np.vstack((putdownFeatures, putdowns))
    # register labels
    pickupLabels = np.hstack((pickupLabels, np.full((1, pickups.shape[0]), label)))
    putdownLabels = np.hstack((putdownLabels, np.full((1, putdowns.shape[0]), label + 1)))
    return (pickupFeatures, pickupLabels, putdownFeatures, putdownLabels)

def trainModels(features, labels):
    # features
    X = features
    # labels
    y = labels.flatten()
    clf = svm.SVC(C=1, kernel='poly', coef0=1.0, probability=True, tol=1e-5)
    clf.fit(X, y)
    return clf

def predict(pickupModel, putdownModel, fileName, fileRange, answerLabel):
    pickupResult = np.array([], dtype=int)
    putdownResult = np.array([], dtype=int)
    pickupCorrect = 0
    putdownCorrect = 0
    for fileNum in fileRange:
        (pickups, putdowns) = featuresFromFile(fileName, fileNum)
        pickupResult = np.append(pickupResult, pickupModel.predict(pickups))
        putdownResult = np.append(putdownResult, putdownModel.predict(putdowns))
    for label in pickupResult:
        if label == answerLabel:
            pickupCorrect += 1
    for label in putdownResult:
            if label == (answerLabel + 1):
                putdownCorrect += 1
    print(pickupResult)
    print(putdownResult)
    print("{0:.2f}".format(100.0 * pickupCorrect / pickupResult.size) + '%')
    print("{0:.2f}".format(100.0 * putdownCorrect / putdownResult.size) + '%')

In [130]:
# 1: pick up chocolate
# 2: put down chocolate
# 3: pick up peach
# 4: put down peach
# 5: pick up pasta
# 6: put down pasta

totalFiles = 16
predictNo = 15

pickupFeatures = np.empty(shape=[0, featureSize])
pickupLabels = np.empty(shape=[1, 0],dtype=int)
putdownFeatures = np.empty(shape=[0, featureSize])
putdownLabels = np.empty(shape=[1, 0],dtype=int)

for fileNum in range(predictNo):
    (pickupFeatures, pickupLabels, putdownFeatures, putdownLabels) = labelFeatures(pickupFeatures, pickupLabels, putdownFeatures, putdownLabels, "chocolate", fileNum, 1)
    (pickupFeatures, pickupLabels, putdownFeatures, putdownLabels) = labelFeatures(pickupFeatures, pickupLabels, putdownFeatures, putdownLabels, "peach", fileNum, 3)
    (pickupFeatures, pickupLabels, putdownFeatures, putdownLabels) = labelFeatures(pickupFeatures, pickupLabels, putdownFeatures, putdownLabels, "pasta", fileNum, 5)

pickupModel = trainModels(pickupFeatures, pickupLabels)
putdownModel = trainModels(putdownFeatures, putdownLabels)

predict(pickupModel, putdownModel, "chocolate", range(predictNo, totalFiles), 1)
predict(pickupModel, putdownModel, "peach", range(predictNo, totalFiles), 3)
predict(pickupModel, putdownModel, "pasta", range(predictNo, totalFiles), 5)

/Users/allenh/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/allenh/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


[1 1 1 1 1 1 1 3 1]
[2 4 2 2 2 4 2 4 2]
88.89%
66.67%
[3 1 3 1 3 3 3 3 1 3 1]
[4 2 2 2 2 2 2 6 4 6 2]
63.64%
18.18%
[5 5 5 3 5 5 5 5 5 5 5]
[6 6 6 6 6 6 6 6 6 6 6]
90.91%
100.00%


In [ ]:
# 124
# 61.36%
# 56.82%
# 31.03%
# 22.41%
# 87.93%
# 82.76%

# 76
# 61.36%
# 56.82%
# 31.03%
# 25.86%
# 87.93%
# 84.48%

# 60
# 61.36%
# 50.00%
# 31.03%
# 24.14%
# 89.66%
# 82.76%

# 8/10
# 72.22%
# 55.56%
# 77.27%
# 68.18%
# 86.36%
# 86.36%
